# Dependências

In [ ]:
# Install required Google Cloud packages (commented out as these are typically one-time setup commands)
# !pip install gcloud
# !gcloud auth application-default login

# Import necessary Python libraries
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical computing
import time                        # Time-related functions
import os                          # Operating system interfaces
import pandas_gbq                  # Pandas integration with BigQuery
from google.cloud import bigquery  # BigQuery client library
import glob                        # File path pattern matching
import openpyxl                    # Excel file handling
import csv                         # CSV file handling
import re                          # Regular expressions

# Note: The actual imports remain exactly as in the original code

# Tratamento 1

In [ ]:
# Load school data for 2024 and 2023 with Latin-1 encoding
df = pd.read_csv('IED_ESCOLAS_2024.csv', sep=',', encoding='latin-1')
df1 = pd.read_csv('IED_ESCOLAS_2023.csv', sep=',', encoding='latin-1')

# Combine both years' data into a single dataframe
tabela_final = pd.concat([df1, df])

# Filter to keep only 2023 and 2024 data
tabela_final = tabela_final[tabela_final['Ano'].isin(['2023', '2024'])]

# Rename columns to standardized names
tabela_final = tabela_final.rename(columns={
    'Ano': 'ano',
    'Região': 'nome_regiao',
    'UF': 'sigla_uf',
    'Código do Município': 'id_municipio',
    'Nome do Município': 'nome_municipio',
    'Código da Escola': 'id_escola',
    'Nome da Escola': 'nome_escola',
    'Localização': 'localizacao',
    'Dependência Administrativa': 'dependencia_adm',
    # Elementary school total groups
    'EF_total_grupo1': 'ef_total_grupo1',
    'EF_total_grupo2': 'ef_total_grupo2',
    'EF_total_grupo3': 'ef_total_grupo3',
    'EF_total_grupo4': 'ef_total_grupo4',
    'EF_total_grupo5': 'ef_total_grupo5',
    'EF_total_grupo6': 'ef_total_grupo6',
    # Elementary school initial years groups
    'EF_iniciais_grupo1': 'ef_iniciais_grupo1',
    'EF_iniciais_grupo2': 'ef_iniciais_grupo2',
    'EF_iniciais_grupo3': 'ef_iniciais_grupo3',
    'EF_iniciais_grupo4': 'ef_iniciais_grupo4',
    'EF_iniciais_grupo5': 'ef_iniciais_grupo5',
    'EF_iniciais_grupo6': 'ef_iniciais_grupo6',
    # Elementary school final years groups
    'EF_finais_grupo1': 'ef_finais_grupo1',
    'EF_finais_grupo2': 'ef_finais_grupo2',
    'EF_finais_grupo3': 'ef_finais_grupo3',
    'EF_finais_grupo4': 'ef_finais_grupo4',
    'EF_finais_grupo5': 'ef_finais_grupo5',
    'EF_finais_grupo6': 'ef_finais_grupo6',
    # High school groups
    'EM_grupo1': 'em_grupo1',
    'EM_grupo2': 'em_grupo2',
    'EM_grupo3': 'em_grupo3',
    'EM_grupo4': 'em_grupo4',
    'EM_grupo5': 'em_grupo5',
    'EM_grupo6': 'em_grupo6'
})

# Replace '--' values with NaN
tabela_final = tabela_final.replace('--', np.nan)

# List of columns to convert to float
transformar_float = [
    'ef_total_grupo1', 'ef_total_grupo2', 'ef_total_grupo3',
    'ef_total_grupo4', 'ef_total_grupo5', 'ef_total_grupo6',
    'ef_iniciais_grupo1', 'ef_iniciais_grupo2', 'ef_iniciais_grupo3',
    'ef_iniciais_grupo4', 'ef_iniciais_grupo5', 'ef_iniciais_grupo6',
    'ef_finais_grupo1', 'ef_finais_grupo2', 'ef_finais_grupo3',
    'ef_finais_grupo4', 'ef_finais_grupo5', 'ef_finais_grupo6',
    'em_grupo1', 'em_grupo2', 'em_grupo3', 'em_grupo4', 'em_grupo5', 'em_grupo6'
]

# Convert specified columns from string to float (handling decimal comma)
for coluna in transformar_float:
    tabela_final[coluna] = tabela_final[coluna].str.replace(',', '.').astype(float)

# Convert year and school ID to integers
tabela_final['ano'] = tabela_final['ano'].astype(int)
tabela_final['id_escola'] = tabela_final['id_escola'].astype(int)

# Tratamento 2

In [ ]:
# Rename columns to use dot notation for easier splitting later
df_trat = tabela_final.rename(columns={
    'ef_total_grupo1': 'ef.total_grupo1',
    'ef_total_grupo2': 'ef.total_grupo2',
    'ef_total_grupo3': 'ef.total_grupo3',
    'ef_total_grupo4': 'ef.total_grupo4',
    'ef_total_grupo5': 'ef.total_grupo5',
    'ef_total_grupo6': 'ef.total_grupo6',
    'ef_finais_grupo1': 'ef.finais_grupo1',
    'ef_finais_grupo2': 'ef.finais_grupo2',
    'ef_finais_grupo3': 'ef.finais_grupo3',
    'ef_finais_grupo4': 'ef.finais_grupo4',
    'ef_finais_grupo5': 'ef.finais_grupo5',
    'ef_finais_grupo6': 'ef.finais_grupo6',
    'ef_iniciais_grupo1': 'ef.iniciais_grupo1',
    'ef_iniciais_grupo2': 'ef.iniciais_grupo2',
    'ef_iniciais_grupo3': 'ef.iniciais_grupo3',
    'ef_iniciais_grupo4': 'ef.iniciais_grupo4',
    'ef_iniciais_grupo5': 'ef.iniciais_grupo5',
    'ef_iniciais_grupo6': 'ef.iniciais_grupo6'
})

# Reshape data from wide to long format
df_trat = df_trat.melt(
    id_vars=['ano', 'nome_regiao', 'sigla_uf', 'id_municipio', 'nome_municipio',
             'id_escola', 'nome_escola', 'localizacao', 'dependencia_adm'],
    value_vars=[
        'ef.total_grupo1', 'ef.total_grupo2', 'ef.total_grupo3',
        'ef.total_grupo4', 'ef.total_grupo5', 'ef.total_grupo6',
        'ef.iniciais_grupo1', 'ef.iniciais_grupo2', 'ef.iniciais_grupo3',
        'ef.iniciais_grupo4', 'ef.iniciais_grupo5', 'ef.iniciais_grupo6',
        'ef.finais_grupo1', 'ef.finais_grupo2', 'ef.finais_grupo3',
        'ef.finais_grupo4', 'ef.finais_grupo5', 'ef.finais_grupo6',
        'em_grupo1', 'em_grupo2', 'em_grupo3', 'em_grupo4', 'em_grupo5', 'em_grupo6'
    ],
    var_name='grupos_docentes_por_ano_escolar',
    value_name='per_valor'
)

df = df_trat

# Split the combined column into two separate columns
df1 = df['grupos_docentes_por_ano_escolar'].str.split('_', n=1, expand=True)
df1.columns = ['ano_escolar', 'docente_grupos']

# Function to transform abbreviated names to full descriptions
def transformar(nome):
    nome = re.sub(r"\bef.total\b", "Ensino Fundamental - Todos os anos", nome)
    nome = re.sub(r"\bef.iniciais\b", "Ensino Fundamental - Anos Iniciais", nome)
    nome = re.sub(r"\bef.finais\b", "Ensino Fundamental - Anos Finais", nome)
    nome = re.sub(r"\bem\b", "Ensino Médio", nome)
    nome = re.sub(r"\bgrupo1\b", "Grupo 1", nome)
    nome = re.sub(r"\bgrupo2\b", "Grupo 2", nome)
    nome = re.sub(r"\bgrupo3\b", "Grupo 3", nome)
    nome = re.sub(r"\bgrupo4\b", "Grupo 4", nome)
    nome = re.sub(r"\bgrupo5\b", "Grupo 5", nome)
    nome = re.sub(r"\bgrupo6\b", "Grupo 6", nome)
    return nome

# Apply transformations to both new columns
df1['ano_escolar'] = df1['ano_escolar'].apply(transformar)
df1['docente_grupos'] = df1['docente_grupos'].apply(transformar)

# Handle special case for 'em' (high school)
df1['ano_escolar'] = df1['ano_escolar'].replace('em', 'Ensino Médio')

# Combine the split columns back with original dataframe
df = pd.concat([df, df1], axis=1)

# Drop the original combined column
df = df.drop(columns='grupos_docentes_por_ano_escolar')

# Rename 'Grupo' to 'Nível' for more professional terminology
df['docente_grupos'] = df['docente_grupos'].replace(to_replace=r'Grupo', value='Nível', regex=True)

# Rename the column to reflect the terminology change
df = df.rename(columns={'docente_grupos': 'docente_niveis'})

# Select final columns for analysis
df = df[['ano', 'sigla_uf', 'localizacao', 'dependencia_adm', 'ano_escolar', 'docente_niveis', 'per_valor']]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 735912 entries, 0 to 735911
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   ano              735912 non-null  int64  
 1   sigla_uf         735912 non-null  object 
 2   localizacao      735912 non-null  object 
 3   dependencia_adm  735912 non-null  object 
 4   ano_escolar      735912 non-null  object 
 5   docente_niveis   735912 non-null  object 
 6   per_valor        492042 non-null  float64
dtypes: float64(1), int64(1), object(5)
memory usage: 39.3+ MB


# Upload

In [ ]:
# Define the BigQuery table schema with Portuguese descriptions
schema = [
    bigquery.SchemaField('ano', 'INTEGER', description='Ano de referência da observação'),
    bigquery.SchemaField('sigla_uf', 'STRING', description='Sigla da Unidade da Federação referente a observação'),
    bigquery.SchemaField('localizacao', 'STRING', description='Se a área da observação é urbana ou rural'),
    bigquery.SchemaField('dependencia_adm', 'STRING',
                       description='Dependência admnistrativa da observação (privada ou pública)'),
    bigquery.SchemaField('ano_escolar', 'STRING',
                       description='Ano escolar referente ao qual docente leciona'),
    bigquery.SchemaField('docente_niveis', 'STRING',
                       description='Cinco categorias de adequação da formação dos docentes em relação à disciplina que leciona'),
    bigquery.SchemaField('per_valor', 'FLOAT',
                       description='Percentual de docentes que lecionam na respectiva agregação em cada um dos níveis da escala do indicador'),
]

# Initialize BigQuery client connection
client = bigquery.Client(project='repositoriodedadosgpsp')

# Create reference to target dataset
dataset_ref = client.dataset('percepcao_mensuracao_qualidade_estado')

# Create reference to target table with standardized naming convention:
# FONTE_algo_intuitivo_dado (INEP_esforco_docente_temporal_v2)
table_ref = dataset_ref.table('INEP_esforco_docente_temporal_v2')

# Configure the load job with our schema definition
job_config = bigquery.LoadJobConfig(schema=schema)

# Execute the load job to upload DataFrame to BigQuery
job = client.load_table_from_dataframe(
    dataframe=df,
    destination=table_ref,
    job_config=job_config
)

# Wait for the job to complete
job.result()